In [1]:
## [滑窗值：合并多少条记录在一起] 【真实结果为 n+1 】
#n_slidingwindow = 100
n_slidingwindow = 10
## [提前n个周期预警, 分钟 = n/2 分钟预警（30s一个间隔）] ，应该小于滑窗值
n_earlywarningcycle = 6

#周期预警的开始位
n_combinrows = n_earlywarningcycle
print('slidingwindow === ' + str(n_slidingwindow) +  ', earlywarningcycle === ' + str(n_earlywarningcycle) )

## 共计58个属性column
n_totalcolumns = 58 

## load csv path
#csv_path = './yiming-arraged/99b6a193b2474c94ba8cf13e7db25612.csv'
csv_path = './yiming-arraged/10000871_deal.csv'
#csv_path = './yiming-arraged/10000871-part_ac.csv'


slidingwindow === 10, earlywarningcycle === 6


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
#from xgboost import XGBRegressor
%matplotlib inline
from matplotlib import pyplot
from sklearn.metrics import mean_absolute_error

import os
from datetime import datetime
from time import gmtime, strftime, sleep

In [3]:
# 预定义的环境变量
s3_prefix_mask = 'xgb-{}-TC{}-SW{}-WC{}-{}' # haierkong-xgb-{filename}-TC_{total columns 58}-SW_{n_slidingwindow 60}-WC_{warningcycle 30}-{created datetime}
#print(s3_prefix_mask) #for testing

csv_path_head, csv_path_tail = os.path.split(csv_path)
#created_date = strftime("%Y_%m_%d_%H_%M_%S", gmtime())
created_date = strftime("%Y%m%d%H%M%S", gmtime())

strfilled_n_slidingwindow=str(n_slidingwindow).rjust(3,'0')
strfilled_n_earlywarningcycle=str(n_earlywarningcycle).rjust(3,'0')
s3_prefix = s3_prefix_mask.format(csv_path_tail, n_totalcolumns, strfilled_n_slidingwindow, strfilled_n_earlywarningcycle, created_date)
print(s3_prefix) #for testing



xgb-10000871_deal.csv-TC58-SW010-WC006-20210126012545


In [4]:
df = pd.read_csv(csv_path,low_memory=False)

#df n_totalcolumns = 58 columns
df = df[[
'1S0Z7F92N86S68KI_1_AUXILIQUIDOPENING',
'1S0Z7F92N86S68KI_1_COMPCURRENT',
'1S0Z7F92N86S68KI_1_COMPEXHAUSTTEMP',
'1S0Z7F92N86S68KI_1_COMPLOAD',
'1S0Z7F92N86S68KI_1_COMPPOWER',
'1S0Z7F92N86S68KI_1_COMPRUNTIME',
'1S0Z7F92N86S68KI_1_COMPSPEED',
'1S0Z7F92N86S68KI_1_COMPSUCTIONTEMP',
'1S0Z7F92N86S68KI_1_COMPVOLTAGE',
'1S0Z7F92N86S68KI_1_CONDSIDEEXHAUSTPRESS',
'1S0Z7F92N86S68KI_1_DISCHARGESUPERHEAT',
'1S0Z7F92N86S68KI_1_ECONPRESS',
'1S0Z7F92N86S68KI_1_ECONREFRTEMP',
'1S0Z7F92N86S68KI_1_EVAPSIDESUCTIONPRESS',
'1S0Z7F92N86S68KI_1_INVERTERTEMP',
'1S0Z7F92N86S68KI_1_MAINFLOWVALVEOPENING',
'1S0Z7F92N86S68KI_1_MAINLOOPLEVEL',
'1S0Z7F92N86S68KI_2_AUXILIQUIDOPENING',
'1S0Z7F92N86S68KI_2_COMPCURRENT',
'1S0Z7F92N86S68KI_2_COMPEXHAUSTTEMP',
'1S0Z7F92N86S68KI_2_COMPLOAD',
'1S0Z7F92N86S68KI_2_COMPPOWER',
'1S0Z7F92N86S68KI_2_COMPRUNTIME',
'1S0Z7F92N86S68KI_2_COMPSPEED',
'1S0Z7F92N86S68KI_2_COMPSUCTIONTEMP',
'1S0Z7F92N86S68KI_2_COMPVOLTAGE',
'1S0Z7F92N86S68KI_2_CONDSIDEEXHAUSTPRESS',
'1S0Z7F92N86S68KI_2_DISCHARGESUPERHEAT',
'1S0Z7F92N86S68KI_2_ECONPRESS',
'1S0Z7F92N86S68KI_2_ECONREFRTEMP',
'1S0Z7F92N86S68KI_2_EVAPSIDESUCTIONPRESS',
'1S0Z7F92N86S68KI_2_INVERTERTEMP',
'1S0Z7F92N86S68KI_2_MAINFLOWVALVEOPENING',
'1S0Z7F92N86S68KI_2_MAINLOOPLEVEL',
'1S0Z7F92N86S68KI_3_AUXILIQUIDOPENING',
'1S0Z7F92N86S68KI_3_COMPCURRENT',
'1S0Z7F92N86S68KI_3_COMPEXHAUSTTEMP',
'1S0Z7F92N86S68KI_3_COMPLOAD',
'1S0Z7F92N86S68KI_3_COMPPOWER',
'1S0Z7F92N86S68KI_3_COMPRUNTIME',
'1S0Z7F92N86S68KI_3_COMPSPEED',
'1S0Z7F92N86S68KI_3_COMPSUCTIONTEMP',
'1S0Z7F92N86S68KI_3_COMPVOLTAGE',
'1S0Z7F92N86S68KI_3_CONDSIDEEXHAUSTPRESS',
'1S0Z7F92N86S68KI_3_DISCHARGESUPERHEAT',
'1S0Z7F92N86S68KI_3_ECONPRESS',
'1S0Z7F92N86S68KI_3_ECONREFRTEMP',
'1S0Z7F92N86S68KI_3_EVAPSIDESUCTIONPRESS',
'1S0Z7F92N86S68KI_3_INVERTERTEMP',
'1S0Z7F92N86S68KI_3_MAINFLOWVALVEOPENING',
'1S0Z7F92N86S68KI_3_MAINLOOPLEVEL',
'SYSTEM_CONDCAPACITY',
'SYSTEM_CONDSIDETEMPIN',
'SYSTEM_CONDSIDETEMPOUT',
'SYSTEM_EVAPCAPACITY',
'SYSTEM_EVAPSIDETEMPOUT',
'SYSTEM_UNITPOWER',
'code','time'
]]

In [5]:
df

,1S0Z7F92N86S68KI_1_AUXILIQUIDOPENING,1S0Z7F92N86S68KI_1_COMPCURRENT,1S0Z7F92N86S68KI_1_COMPEXHAUSTTEMP,1S0Z7F92N86S68KI_1_COMPLOAD,1S0Z7F92N86S68KI_1_COMPPOWER,1S0Z7F92N86S68KI_1_COMPRUNTIME,1S0Z7F92N86S68KI_1_COMPSPEED,1S0Z7F92N86S68KI_1_COMPSUCTIONTEMP,1S0Z7F92N86S68KI_1_COMPVOLTAGE,1S0Z7F92N86S68KI_1_CONDSIDEEXHAUSTPRESS,...,1S0Z7F92N86S68KI_3_MAINFLOWVALVEOPENING,1S0Z7F92N86S68KI_3_MAINLOOPLEVEL,SYSTEM_CONDCAPACITY,SYSTEM_CONDSIDETEMPIN,SYSTEM_CONDSIDETEMPOUT,SYSTEM_EVAPCAPACITY,SYSTEM_EVAPSIDETEMPOUT,SYSTEM_UNITPOWER,code,time
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1876097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1876098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1876099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#数据清理0： 排序
# 按照时间排序数组后，然后删除时间的属性
df['time']=pd.to_datetime(df['time'])
df.sort_values('time', inplace=True)
# save for time sort checking...
#df.to_csv('modeldata_sortbytime.csv', header=True, index=True) #for testing
df = df.drop(['time'], axis=1)

In [7]:
#数据清理0: 转化 code 到 hascode = true/false 
# 根据hascode属性的请看，添加Ishascode =true/false 的列
df['hascode'] = (np.where(df['code'].isnull().values, False, True)).astype(object)
df = df.drop(['code'], axis=1)
#df.info()  # for testing
#df.head(3)  # for testing

##数据清理0:  将 “Ishascode =true/false”的列置换到第一列，作为分类Lable
model_data = pd.get_dummies(df)
model_data = pd.concat([model_data['hascode_True'], model_data.drop(
    ['hascode_False', 'hascode_True'], axis=1)], axis=1)
#willdel#model_data_values = model_data_values[0:10, 13:15] ## for testing
#willdel#DataFrame(model_data_values).info()

In [8]:
model_data

,hascode_True,1S0Z7F92N86S68KI_1_AUXILIQUIDOPENING,1S0Z7F92N86S68KI_1_COMPCURRENT,1S0Z7F92N86S68KI_1_COMPEXHAUSTTEMP,1S0Z7F92N86S68KI_1_COMPLOAD,1S0Z7F92N86S68KI_1_COMPPOWER,1S0Z7F92N86S68KI_1_COMPRUNTIME,1S0Z7F92N86S68KI_1_COMPSPEED,1S0Z7F92N86S68KI_1_COMPSUCTIONTEMP,1S0Z7F92N86S68KI_1_COMPVOLTAGE,...,1S0Z7F92N86S68KI_3_EVAPSIDESUCTIONPRESS,1S0Z7F92N86S68KI_3_INVERTERTEMP,1S0Z7F92N86S68KI_3_MAINFLOWVALVEOPENING,1S0Z7F92N86S68KI_3_MAINLOOPLEVEL,SYSTEM_CONDCAPACITY,SYSTEM_CONDSIDETEMPIN,SYSTEM_CONDSIDETEMPOUT,SYSTEM_EVAPCAPACITY,SYSTEM_EVAPSIDETEMPOUT,SYSTEM_UNITPOWER
147490,1,100.0,93.0,36.8,62.0,NaN,2097.0,21284.0,5.6,375.0,...,346.6,0.0,0.0,62.0,NaN,25.1,27.9,NaN,10.6,NaN
147495,1,100.0,93.0,36.9,NaN,NaN,2097.0,21163.0,6.0,374.0,...,346.9,0.0,0.0,62.0,NaN,25.1,27.9,NaN,10.6,111.0
155193,1,100.0,78.0,36.5,NaN,NaN,2172.0,20393.0,6.3,389.0,...,345.7,0.0,0.0,57.0,NaN,NaN,NaN,NaN,NaN,NaN
155196,1,100.0,77.0,36.5,NaN,NaN,2172.0,20383.0,6.4,389.0,...,346.0,0.0,0.0,57.0,NaN,26.4,27.6,NaN,10.6,NaN
327473,1,0.0,0.0,29.3,0.0,0.0,2945.0,0.0,28.6,388.0,...,268.4,0.0,NaN,56.0,NaN,29.0,30.4,NaN,9.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876096,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1876097,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1876098,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1876099,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# 定义函数 #

## transform the timeseries data into supervised learning
#时间序列数据集转换为监督学习问题，将《多列时间序列数据》转换为《监督学习问题》
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
       n_vars = 1 if type(data) is list else data.shape[1]
       df = DataFrame(data)
       cols = list()
        
       # input sequence (t-n, ... t-1)
       for i in range(n_in, 0, -1):
              cols.append(df.shift(i))
                
       # forecast sequence (t, t+1, ... t+n)
       for i in range(0, n_out):
              cols.append(df.shift(-i))
                
       # put it all together
       agg = concat(cols, axis=1)
        
       # drop rows with NaN values
       if dropnan:
              agg.dropna(inplace=True)
       return agg.values


# 分段滑窗，避免内存溢出； 安装n进行滑窗，缺省splite行为5W
# splite_md = 行分段的数量，分的越小，内存占用越小
def splite_series_to_supervised(data, n_in=1, splite_md=500000):
    splited_series_to_supervised = pd.DataFrame()  #定义一个临时的dataframe，用于解决内存溢出的问题
    # start, stop, step 三个参数可以为负数
    for i in range(0,len(data),splite_md):
        print('/ total= '+str(len(data))+', now splited_working_number= ' + str(i)), # for testing    
        splited_series_to_supervised = splited_series_to_supervised.append(DataFrame(
            series_to_supervised(model_data.iloc[ i : i+splite_md ], n_in=n_in, dropnan=False)))
        #print(model_data.iloc[ i : i+splite_md ].info()) # for testing
    
    return splited_series_to_supervised


#滑窗后处理：滑窗后的数据清理，将不是最后错误之前发生的滑窗全部删除
# 用 n_slidingwindow（滑窗数量） 做一个循环：不考虑（查询）最后一位的hascode，前面所有hascode=1 (每间隔58个列的第一个是hascode=true/false)的行全部删除；
# 即：只保留全部hascode=0的行(没有错误发生的行) 与 最后一位hascode=1的行(第一个错误发生的行)
def clear_supervised(data, n_slidingwindow, n_totalcolumns):
    count = 0
    while count < n_slidingwindow :
        n_colnum = count * n_totalcolumns
        data.drop(index=data[data[n_colnum].isin([True])].index, inplace=True)
        print('/ total = '+str(n_slidingwindow) +', now working on = ' + str(count)), # for testing
        #print('column num= '+ str(n_colnum)) # for testing
        count = count + 1
    
    return data

#-----------------------#-----------------------#-----------------------
# 滑窗后处理：正确滑窗，应该是 “有错和无错，各自一条”；
#即：挑出报错时的最后一条数据 + 删除上面N条未报错数据（上一条正常数据为行数为 except - n）
#适用于pd.sample随机
def pickup_supervised_4train_v01(data, n_slidingwindow, n_totalcolumns, splite_md=500000):
    splited_series_to_supervised = pd.DataFrame()  #定义一个临时的dataframe，用于解决内存溢出的问题
    
    for i in range(0,len(data),splite_md):
        print('\nworking_number ==' + str(i)), # for testing    

        splited_data= data.iloc[ i : i+splite_md ]
        splited_data.reset_index(drop=True,inplace=True)
        print('/ splited_data ==== '+str(splited_data.shape[0])),

        n_checkpoint = n_totalcolumns * n_slidingwindow # 检查点位数 
        index_hascode_truerows = splited_data[splited_data[n_checkpoint].isin([True])].index #检查点列 为真的Index号，用于下一步挑出来
        print('/ index_hascode_truerows ====== '+str(index_hascode_truerows)),

        target_data = pd.DataFrame(data=splited_data,index=index_hascode_truerows) # 把检查点列 为真 挑出来
        target_data = target_data.append(pd.DataFrame(data=splited_data,index=index_hascode_truerows-n_slidingwindow)) # 【-滑窗数量可以取到完全不同的validi值，适用于sample随机】把检查点列 为真的上n行，挑出来 (N等于滑窗个数)
        target_data[n_checkpoint].fillna(0, inplace=True) #上一行的 检查点列 有可能是空的，空值清洗为0
        print('/ target_data ======== '+ str(target_data.shape[0])),

        splited_series_to_supervised = splited_series_to_supervised.append(target_data)

    print('\n========================================')
    print('splited_series_to_supervised  ============================== '+ str(splited_series_to_supervised.shape[0])) # for testing1
    splited_series_to_supervised.drop_duplicates(inplace=True) #清除重复的行（造成1增加）
    return splited_series_to_supervised

#适用于pd.sample随机
def pickup_supervised_4train_v02(data, n_slidingwindow, n_totalcolumns, splite_md=500000):
    splited_series_to_supervised = pd.DataFrame()  #定义一个临时的dataframe，用于解决内存溢出的问题
    
    for i in range(0,len(data),splite_md):
        print('\nworking_number ==' + str(i)), # for testing    

        splited_data= data.iloc[ i : i+splite_md ]
        splited_data.reset_index(drop=True,inplace=True)
        print('/ splited_data ==== '+str(splited_data.shape[0])),

        n_checkpoint = n_totalcolumns * n_slidingwindow # 检查点位数 
        index_hascode_truerows = splited_data[splited_data[n_checkpoint].isin([True])].index #检查点列 为真的Index号，用于下一步挑出来
        print('/ index_hascode_truerows ====== '+str(index_hascode_truerows)),

        target_data = pd.DataFrame(data=splited_data,index=index_hascode_truerows) # 把检查点列 为真 挑出来
        
          #--- 以状态位=1的row为准，向上画出一个状态位=0的矩阵 ---# 
        for i in range(1,n_slidingwindow+1): 
            target_data = target_data.append(pd.DataFrame(data=splited_data,index=index_hascode_truerows-i)) # 把检查点列 为真的上n滑窗行，挑出来 (N等于滑窗个数)，不适用于sample随机
            #print(' DivRows=' + str(i)), #for testing
            
        target_data[n_checkpoint].fillna(0, inplace=True) #上一行的 检查点列 有可能是空的，空值清洗为0
        print('/ target_data ======== '+ str(target_data.shape[0])),

        splited_series_to_supervised = splited_series_to_supervised.append(target_data)

    print('\n========================================')
    print('splited_series_to_supervised  ============================== '+ str(splited_series_to_supervised.shape[0])) # for testing1
    splited_series_to_supervised.drop_duplicates(inplace=True) #清除重复的行（造成1增加）    
    return splited_series_to_supervised

#-----------------------#-----------------------#-----------------------


# 滑窗后处理：准备xgboost的Lable数据集，处理：
# 1）将最后的hasissuce code lable放入第一列； 
# 2）删除合并后最后一行row的信息（共计58个属性column）
def ready4inference_supervised(data, n_slidingwindow, n_totalcolumns, n_combinrows):
    #两个工作：1）把最后的一位 hascode=true/false 挪到了第一位作为lable； 2）删除最后的58个属性列；    
    n_lasthascodepoint = n_slidingwindow*n_totalcolumns
    print('n_lasthascodepoint=' + str(n_lasthascodepoint))
    
    n_drop_end = (n_slidingwindow + 1) * n_totalcolumns # 需要增加58个column才是end；
    print('n_drop_end=' + str(n_drop_end))
    
    n_drop_start = (n_slidingwindow - n_combinrows) * n_totalcolumns
    print('n_drop_start=' + str(n_drop_start))
    
    data = pd.concat([data[n_lasthascodepoint], 
                            data.drop(data.iloc[:, n_drop_start:n_drop_end], axis=1)], axis=1) 
    print('\nFINALL data  ============================== '+ str(data.shape[0])) # for testing

    return data

# inference预测处理：传入modeldata数据和 sagemaker inference handle，获得预测结果
def sagemaker_predict(data, xgb_predictor, rows=1000):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join(
            [predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')   

In [10]:
#数据滑窗 1 ： 分段滑窗，避免内存溢出
## transform the timeseries data into supervised learning
#旧方法存档#model_data = DataFrame(series_to_supervised(model_data, n_in=n_slidingwindow, dropnan=False))  #合并n行到一行
#旧方法存档#model_data.info()   
model_data = splite_series_to_supervised(model_data, n_slidingwindow, 50000)

#model_data.to_csv('spliteseriestosupervised_data.csv', header=False, index=False) # has NO header for xgboost bin:log
model_data.info() #for testing

/ total= 1876101, now splited_working_number= 0
/ total= 1876101, now splited_working_number= 50000
/ total= 1876101, now splited_working_number= 100000
/ total= 1876101, now splited_working_number= 150000
/ total= 1876101, now splited_working_number= 200000
/ total= 1876101, now splited_working_number= 250000
/ total= 1876101, now splited_working_number= 300000
/ total= 1876101, now splited_working_number= 350000
/ total= 1876101, now splited_working_number= 400000
/ total= 1876101, now splited_working_number= 450000
/ total= 1876101, now splited_working_number= 500000
/ total= 1876101, now splited_working_number= 550000
/ total= 1876101, now splited_working_number= 600000
/ total= 1876101, now splited_working_number= 650000
/ total= 1876101, now splited_working_number= 700000
/ total= 1876101, now splited_working_number= 750000
/ total= 1876101, now splited_working_number= 800000
/ total= 1876101, now splited_working_number= 850000
/ total= 1876101, now splited_working_number= 900000

In [11]:
model_data

,0,1,2,3,4,5,6,7,8,9,...,628,629,630,631,632,633,634,635,636,637
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,346.6,0.0,0.0,62.0,NaN,25.1,27.9,NaN,10.6,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,346.9,0.0,0.0,62.0,NaN,25.1,27.9,NaN,10.6,111.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,345.7,0.0,0.0,57.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,346.0,0.0,0.0,57.0,NaN,26.4,27.6,NaN,10.6,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,268.4,0.0,NaN,56.0,NaN,29.0,30.4,NaN,9.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26096,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26097,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26098,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26099,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# 滑窗后数据处理 2，清理不符合要求的滑入数据；
model_data = clear_supervised(model_data, n_slidingwindow, n_totalcolumns)
#model_data.to_csv('clearsupervised_data.csv', header=False, index=False) # has NO header for xgboost bin:log

# 校验查询用
n_checkpoint = n_totalcolumns * n_slidingwindow # 检查点位数  
print('\nAfter clear_supervised model_data issue rows on point = ' + str(n_checkpoint))
model_data[model_data[n_checkpoint].isin([True])].info()  ## 校验查询用 #for testing
print('\nAfter clear_supervised model_data total  ============================== ')
model_data.info()  ## 校验查询用 #for testing
model_data

/ total = 10, now working on = 0
/ total = 10, now working on = 1
/ total = 10, now working on = 2
/ total = 10, now working on = 3
/ total = 10, now working on = 4
/ total = 10, now working on = 5
/ total = 10, now working on = 6
/ total = 10, now working on = 7
/ total = 10, now working on = 8
/ total = 10, now working on = 9

After clear_supervised model_data issue rows on point = 580
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Columns: 638 entries, 0 to 637
dtypes: float64(638)
memory usage: 5.0 KB

After clear_supervised model_data total  ============================== 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1872111 entries, 0 to 26100
Columns: 638 entries, 0 to 637
dtypes: float64(638)
memory usage: 8.9 GB


,0,1,2,3,4,5,6,7,8,9,...,628,629,630,631,632,633,634,635,636,637
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,346.6,0.0,0.0,62.0,NaN,25.1,27.9,NaN,10.6,NaN
106,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26096,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26097,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26098,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26099,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#滑窗后数据处理 3： 分段挑选有效数据，避免内存溢出
model_data = pickup_supervised_4train_v02(model_data,n_slidingwindow, n_totalcolumns, splite_md=50000)
#model_data.to_csv('pickupsmallsupervisedfortrain_data.csv', header=False, index=False) # has NO header for xgboost bin:log

print('\nFINALL clear_supervised model_data issue rows on point = ' + str(n_checkpoint))
model_data[model_data[n_checkpoint].isin([True])].info()  ## 校验查询用 #for testing
print('\nFINALL model_data  ============================== '+ str(model_data.shape[0])) # for testing
model_data.info()  ## 校验查询用 #for testing
model_data


working_number ==0
/ splited_data ==== 50000
/ index_hascode_truerows ====== Int64Index([0], dtype='int64')
/ target_data ======== 11

working_number ==50000
/ splited_data ==== 50000
/ index_hascode_truerows ====== Int64Index([], dtype='int64')
/ target_data ======== 0

working_number ==100000
/ splited_data ==== 50000
/ index_hascode_truerows ====== Int64Index([], dtype='int64')
/ target_data ======== 0

working_number ==150000
/ splited_data ==== 50000
/ index_hascode_truerows ====== Int64Index([], dtype='int64')
/ target_data ======== 0

working_number ==200000
/ splited_data ==== 50000
/ index_hascode_truerows ====== Int64Index([], dtype='int64')
/ target_data ======== 0

working_number ==250000
/ splited_data ==== 50000
/ index_hascode_truerows ====== Int64Index([], dtype='int64')
/ target_data ======== 0

working_number ==300000
/ splited_data ==== 50000
/ index_hascode_truerows ====== Int64Index([], dtype='int64')
/ target_data ======== 0

working_number ==350000
/ splited_dat

,0,1,2,3,4,5,6,7,8,9,...,628,629,630,631,632,633,634,635,636,637
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,346.6,0.0,0.0,62.0,NaN,25.1,27.9,NaN,10.6,NaN
-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#滑窗后数据处理 4：准备xgboost training lable的标志位
model_data = ready4inference_supervised(model_data,n_slidingwindow, n_totalcolumns,n_combinrows)
#model_data.to_csv('ready4inference_data.csv', header=True, index=True)  #for testing
model_data.info()  ## 校验查询用 #for testing

n_lasthascodepoint=580
n_drop_end=638
n_drop_start=232

FINALL data  ============================== 2
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to -1
Columns: 233 entries, 580 to 231
dtypes: float64(233)
memory usage: 3.7 KB


In [15]:
# for testing
#model_data.columns
#model_data.columns.values
#model_data.head(3)
model_data

,580,0,1,2,3,4,5,6,7,8,...,222,223,224,225,226,227,228,229,230,231
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
## train_data, validation_data = np.split(model_data.sample(frac=1,random_state=1729), [int(0.7 * len(model_data))])  #随机的sample，train/vali = 70/30比例
train_data, validation_data = np.split(model_data, [int(0.8 * len(model_data))])  ##顺序的sample，train/vali = 70/30比例
#train_data.to_csv('train_data.csv', header=True, index=False) #has header #header for autoML on sagemaker
train_data.to_csv('train_data.csv', header=False, index=False) # has NO header for xgboost bin:log
validation_data.to_csv('validation_data.csv', header=False, index=False)

whole_data = model_data
whole_data.to_csv('whole_data.csv', header=False, index=False)

In [17]:
!ls -lht *.csv

-rw-r--r-- 1 root root 472 Jan 26 01:28 whole_data.csv
-rw-r--r-- 1 root root 236 Jan 26 01:28 validation_data.csv
-rw-r--r-- 1 root root 236 Jan 26 01:28 train_data.csv


In [18]:
import os
import boto3
import sagemaker

sess = sagemaker.Session()

bucket = sess.default_bucket()
# s3_prefix = 'haier-xgb-{}-SW_{}-WC_{}-{}' #for testing

boto3.Session().resource('s3').Bucket(bucket).Object(
    os.path.join(s3_prefix, 'train/train_data.csv')).upload_file('train_data.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(
    s3_prefix, 'validation/validation_data.csv')).upload_file('validation_data.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(
    s3_prefix, 'wholedata/whole_data.csv')).upload_file('whole_data.csv')


In [19]:
print('upload completed in: ' + s3_prefix)
print('upload completed in bucket: ' +bucket)

upload completed in: xgb-10000871_deal.csv-TC58-SW010-WC006-20210126012545
upload completed in bucket: sagemaker-cn-northwest-1-337575217701


---
## Amazon SageMaker 实验 （Experiments） - 使用Studio高效观察同一个问题，不同训练任务的指标对比 ###
* 创建实验集合 sagemaker_experiment

# 设置 IAM 角色
from sagemaker import get_execution_role

role = get_execution_role()
# 获取 XGBoost Docker 镜像
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost')
display(container)

### Amazon SageMaker 实验 （Experiments） - 使用Studio高效观察同一个问题，不同训练任务的指标对比 ###
#创建实验集合 sagemaker_experiment

create_date = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
sagemaker_experiment = Experiment.create(experiment_name="sm-bowl-{}".format(create_date), 
                                              description="Using xgboost to predict by YM", 
                                              sagemaker_boto_client=boto3.client('sagemaker'))

# 单个Trail，添加入Experiment
trial = Trial.create(trial_name="sm-single-trial-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime())), 
                     experiment_name=sagemaker_experiment.experiment_name,
                     sagemaker_boto_client=boto3.client('sagemaker'))


# 超参数
xgb_set_hyperparameters = {"max_depth":5,
                        "eta":0.2,
                        "gamma":4,
                        "min_child_weight":6,
                        "subsample":0.8,
                        "silent":0,
                        "objective":'binary:logistic',
                        "num_round":100}

# 创建 SageMaker 评估程序对象
import sagemaker
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role,
                                    hyperparameters=xgb_set_hyperparameters,
                                    train_instance_count=1,
                                    train_instance_type='ml.m5.12xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, s3_prefix),
                                    base_job_name="sm-xgboost-jobs", # sagemaker jobs
                                    sagemaker_session=sess)

# 配置数据输入
s3_input_train = sagemaker.s3_input(
    s3_data='s3://{}/{}/train'.format(bucket, s3_prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(
    s3_data='s3://{}/{}/validation/'.format(bucket, s3_prefix), content_type='csv')

#单个Trail，添加入Experiment # 训练模型
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation},
        experiment_config={
            "ExperimentName": sagemaker_experiment.experiment_name, 
            "TrialName": trial.trial_name,
            "TrialComponentDisplayName": "Training",})


### Amazon SageMaker 实验 （Experiments） - 使用Studio高效观察同一个问题，不同训练任务的指标对比 ###
#同时启动多个Trail，均加入Experiment

min_child_weights = [1, 5, 8, 10]

for weight in min_child_weights:
    xgb_set_hyperparameters["min_child_weight"] = weight
    trial = Trial.create(trial_name="sm-hyper-trial-{}-weight-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()), weight), 
                         experiment_name=sagemaker_experiment.experiment_name,
                         sagemaker_boto_client=boto3.client('sagemaker'))

    temp_xgb = sagemaker.estimator.Estimator(container,
                                          role,
                                          hyperparameters=xgb_set_hyperparameters,
                                          train_instance_count=1,
                                          train_instance_type='ml.m5.12xlarge',
                                          output_path='s3://{}/{}/output'.format(bucket, s3_prefix),
                                          base_job_name="sm-xgboost-jobs",
                                          sagemaker_session=sess)

    temp_xgb.fit({'train': s3_input_train, 'validation': s3_input_validation},
                wait=False,
                experiment_config={
                    "ExperimentName": sagemaker_experiment.experiment_name, 
                    "TrialName": trial.trial_name,
                    "TrialComponentDisplayName": "Training",})

# 部署模型
Variant = 'AllTraffic' # 配置默认部署变体
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m5.4xlarge')

from sagemaker.predictor import csv_serializer
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

#定义一个函数，给下面调用
def predict(data, rows=50):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join(
            [predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

test_data= model_data

#调用函数
predictions = predict(test_data.to_numpy()[:, 1:]) #所有的行，从第一列开始（忽略0列）

y_labels = test_data.iloc[:, 0]
y_scores = predictions

#结果 1：测试数据集结果 与 预测结果 的数量对比
print('test_data: 1='+ str((y_labels == 1).sum()) + ', 0='+str((y_labels == 0).sum()) +' / '),
print('predictions Result: 1='+ str((np.round(y_scores) == 1).sum()) + ', 0='+str((np.round(y_scores) == 0).sum()))

#结果 2：交叉表，说明 测试数据集结果 与 预测结果 的正确情况对比
display(pd.crosstab(index=test_data.iloc[:, 0], columns=np.round(predictions), 
            rownames=['test_data'], colnames=['predictions_data'],margins=False))
#print('\n')

#结果 4：ROC图，越大越好，无限接近与1
from sklearn.metrics import roc_curve, auc                     #导入库
fpr, tpr, thresholds  =  roc_curve(y_labels, y_scores)  #计算真正率和假正率
roc_auc = auc(fpr,tpr)                                         #计算auc的值
plt.figure()
lw = 3
plt.figure(figsize=(5,5))
plt.plot(fpr, tpr, color='darkorange',lw=lw, label='ROC curve (area = %0.2f)' % roc_auc) #假正率为横坐标，真正率为纵坐标做曲线
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

#结果 3：AUC值，越大越好，无限接近与1
import numpy as np
from sklearn.metrics import roc_auc_score
##y_labels=[0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1.
##y_scores=[4.92438182e-01 1.74370125e-01 8.58997216e-01 1.30175965e-01 
# 调用sklearn中的方法计算AUC，与后面自己写的方法作对比
print('sklearn AUC:', roc_auc_score(y_labels, y_scores))
#打印全局变量值
print('\nTraining Mode: n_slidingwindow = '+ str(n_slidingwindow)),
print('and n_earlywarningcycle = '+ str(n_earlywarningcycle)),
print('\ncsv_path = '+ str(csv_path)),
print('\nxgb_predictor.endpoint = ' + str(xgb_predictor.endpoint)),

#结果 5：分布柱状图；用于分析和优化结果（round函数）；
plt.hist(predictions, bins=10, range=(0, 1))
plt.show()

#用于分析和优化结果（round函数）；
round_predictions=0.45
pd.crosstab(index=test_data.iloc[:, 0], columns=np.where(predictions > round_predictions, 1, 0), 
            rownames=['test_data'], colnames=['predictions_data'],margins=False)


print(xgb_predictor.endpoint)

test_data.info()
test_data[test_data[n_checkpoint].isin([True])].info()  ## 校验查询用 #for testing

!pip show sagemaker